# Day 14: Reindeer Olympics

[*Advent of Code 2015 day 14*](https://adventofcode.com/2015/day/14) and [*solution megathread*](https://www.reddit.com/3wqtx2)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/14/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F14%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


In [2]:
import sys

print(sys.executable)

/Users/cj/Documents/advent-of-code/venv/bin/python


## Part One

In [3]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [4]:
%load_ext pycodestyle_magic

In [5]:
%pycodestyle_on

## Comments

...

In [6]:
testdata = """Comet can fly 14 km/s for 10 seconds, but then must rest for 127 seconds.
Dancer can fly 16 km/s for 11 seconds, but then must rest for 162 seconds."""

inputdata = downloaded['input']

1:80: E501 line too long (87 > 79 characters)


In [7]:
inputdata[:100]

'Rudolph can fly 22 km/s for 8 seconds, but then must rest for 165 seconds.\nCupid can fly 8 km/s for '

In [8]:
import re


def parse_data(data: str):
    output = []
    pattern = re.compile(
        r'^(?P<deer>\w+) can fly (?P<speed>\d+) km/s for (?P<endurance>\d+) ' +
        r'seconds, but then must rest for (?P<holdoff>\d+) seconds\.$')
    for line in data.splitlines():
        matches = pattern.match(line)
        perf = matches.groupdict()
        for intfield in ['speed', 'endurance', 'holdoff']:
            perf[intfield] = int(perf[intfield])
        output.append(perf)
    return output


def performance_to_state(perf, debug=False):
    if debug:
        print(perf)
    return perf | {
        'location': 0,
        'holdoff_r': 0,
        'endurance_r': perf['endurance']}


def update_state(state, delta=1, debug=False):
    if state['holdoff_r'] > 0:
        if state['holdoff_r'] <= delta:
            delta_r = delta - state['holdoff_r']
            if debug:
                print(f'case 1: {state["deer"]} has rested ' +
                      f'{state["holdoff_r"]} seconds, now racing ' +
                      f'another {delta_r} seconds')
            state['holdoff_r'] = 0
            state['endurance_r'] = state['endurance']
            if delta_r > 0:
                update_state(state, delta_r, debug)
            return
        state['holdoff_r'] -= delta
        if debug:
            print(f'case 2: {state["deer"]} is resting another {delta} ' +
                  f'seconds, {state["holdoff_r"]} seconds rest remaining')
        return
    elif state['endurance_r'] > delta:
        state['location'] += delta * state['speed']
        state['endurance_r'] -= delta
        if debug:
            print(f'case 3: {state["deer"]} raced {delta} seconds, now in ' +
                  f'location {state["location"]} with a remaining endurance ' +
                  f'of {state["endurance_r"]}')
        return
    else:
        delta_t = state['endurance_r']
        delta_n = delta - delta_t
        state['endurance_r'] = 0
        state['location'] += delta_t * state['speed']
        state['holdoff_r'] = state['holdoff']
        if debug:
            print(f'case 4: {state["deer"]} raced {delta_t} seconds, now in ' +
                  f'location {state["location"]}, will rest for ' +
                  f'{state["holdoff_r"]} with a remaining {delta_n} ' +
                  'seconds to race')
        update_state(state, delta_n, debug)
        return


def race_deer(states, duration=1000, debug=False):
    for state in states:
        update_state(state, delta=duration, debug=debug)
    return [{"deer": state["deer"],
            "location": state["location"]} for state in states]


def my_part1_solution(data, duration=1000, debug=False):
    states = [performance_to_state(p, debug) for p in parse_data(data)]
    return race_deer(states, duration=duration, debug=debug)

In [9]:
my_part1_solution(testdata, duration=1000, debug=False)

[{'deer': 'Comet', 'location': 1120}, {'deer': 'Dancer', 'location': 1056}]

In [10]:
results = my_part1_solution(inputdata, duration=2503, debug=False)
print(results)
print(max(result["location"] for result in results))

[{'deer': 'Rudolph', 'location': 2640}, {'deer': 'Cupid', 'location': 2696}, {'deer': 'Prancer', 'location': 2484}, {'deer': 'Donner', 'location': 2550}, {'deer': 'Dasher', 'location': 2508}, {'deer': 'Comet', 'location': 2520}, {'deer': 'Blitzen', 'location': 2592}, {'deer': 'Vixen', 'location': 2560}, {'deer': 'Dancer', 'location': 2527}]
2696


In [11]:
HTML(downloaded['part1_footer'])

## Part Two

In [12]:
HTML(downloaded['part2'])

In [13]:
def score_states(states, debug=False):
    lead_location = max(state["location"] for state in states)
    for state in states:
        if state["location"] == lead_location:
            state["score"] += 1
            if debug:
                print(f'{state["deer"]} scored this second for ' +
                      f'a score of {state["score"]}')


def my_part2_solution(data, duration=1000, debug=False):
    states = [performance_to_state(p, debug) for p in parse_data(data)]
    for state in states:
        state["score"] = 0
    for _ in range(duration):
        _ = race_deer(states, duration=1, debug=debug)
        score_states(states, debug)
    if debug:
        print(states)
    return max(state["score"] for state in states)

In [14]:
my_part2_solution(testdata, duration=1000, debug=False)

689

In [15]:
my_part2_solution(inputdata, duration=2503, debug=False)

1084

In [16]:
HTML(downloaded['part2_footer'])

Looking at the solution megathread, ["fatpollo"](https://www.reddit.com/r/adventofcode/comments/3wqtx2/comment/cxydr39/?utm_source=share&utm_medium=web2x&context=3) in particular had an elegant solution using [itertools.cycle](https://docs.python.org/3/library/itertools.html#itertools.cycle) and [itertools.accumulate](https://docs.python.org/3/library/itertools.html#itertools.accumulate). I currently have some issue with my notebook output, but even though timeit indicated a duration of some tens of milliseconds, the itertools solution again is vastly more elegant, so I make a stab at using it, my style

In [17]:
from collections import defaultdict, Counter
from itertools import cycle, accumulate


def my_part2_solution_itertools(data, duration=1000, debug=False):
    performances = parse_data(data)
    history = defaultdict(list)
    for performance in performances:
        steps = cycle(
            [performance["speed"]] * performance["endurance"] +
            [0]*performance["holdoff"])
        history[performance["deer"]] = list(
            accumulate(next(steps) for _ in range(duration)))
    by_distance = max(h[-1] for h in history.values())
    if debug:
        print(f'{by_distance=}')
    all_steps = zip(*history.values())
    scored = []
    for step in all_steps:
        for i, distance in enumerate(step):
            if distance == max(step):
                scored.append(i)
    return max(Counter(scored).values())

In [18]:
my_part2_solution_itertools(testdata, duration=1000, debug=False)

689

Interestingly, this is off by one, perhaps because at some point both deer scored? I build the `scored` list differently than in the solution mentioned, because I'm not sure how `Counter` could work otherwise... _(Indeed, the two dimensional list comprehension created a one-dimensional list which I could replicate with two for loops rather than just one)_

In [19]:
my_part2_solution_itertools(inputdata, duration=2503, debug=False)

1084